In [ ]:
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import tensorflow as tf
import keras
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import nltk
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

In [ ]:
def read_dataset():
    df = pd.read_csv("SentCollection - HOBBIT.csv", encoding='iso-8859-9')

    nltk.download('stopwords')
    stop_word_list = stopwords.words('english')

    # data = data.dropna().reset_index(drop=True)
    df.dropna().reset_index(drop=True)
    df.isnull().sum() # Eksik değerlerin sayısını döndürür.

    # print(len(df.columns))
    X = df[df.columns[0:1]].values
    y = df[df.columns[1]]

    Y = one_hot_encode(y)
    print(X.shape)
    return (X, Y)

    data.dropna().reset_index(drop = True)
    data.isnull().sum()

In [ ]:
def preprocess_text(conv):

    sentence = re.sub('[\d\s]', ' ', str(conv))
    sentence = re.sub('[^\w\s]', ' ', str(sentence))
    sentence = re.sub(r'\s+', ' ', sentence)
    sentence = re.sub(r"\b[\w\s]\b", ' ',str(sentence))
        
    WPT = nltk.WordPunctTokenizer()
    tokens = WPT.tokenize(sentence)
    filtered_tokens = [token for token in tokens if token not in stop_word_list]
    single_doc = ' '.join(filtered_tokens)
    
    return single_doc.lower()

    words = []
    for i in X:
        words.append(i.split())

    tokenizer_obj = Tokenizer()
    tokenizer_obj.fit_on_texts(X)
    X = tokenizer_obj.texts_to_sequences(X)
    X = pad_sequences(X)

    scaler = StandardScaler()
    X= scaler.fit_transform(X)

In [ ]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels, n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [ ]:
X, Y = read_dataset()

X, Y = shuffle(X, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.20, random_state=0)

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

learning_rate = 0.3
training_epochs = 100
data_history = np.empty(shape=[1], dtype=float)
n_dim = X.shape[1]
print("n_dim", n_dim)
n_class = 2
model_path = "C:/Users/.../BankNotes"

n_hidden_1 = 4
n_hidden_2 = 4
n_hidden_3 = 4
n_hidden_4 = 4

x = tf.placeholder(tf.float32, [None, n_dim])
W = tf.Variable(tf.zeros([n_dim, n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32, [None, n_class])

In [ ]:
def multilayer_perceptron(x, weights, biases):

    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)

    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.sigmoid(layer_4)

    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

In [ ]:
weights = {
    'h1': tf.Variable(tf.truncated_normal([n_dim, n_hidden_1])),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_4, n_class]))
}
biases = {
    'b1': tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2': tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3': tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4': tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_class]))
}

In [ ]:
init = tf.global_variables_initializer()

saver = tf.train.Saver()

y = multilayer_perceptron(x, weights, biases)

data_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(data_function)

sess = tf.Session()
sess.run(init)
train_x = test_y.astype(float) /255.
train_y = train_y.astype(float) /255.
train_x = train_x.astype(float) /255.
train_y = train_x.astype(float) /255.

In [ ]:
mse_history = []
accuracy_history = []

for epoch in range(training_epochs):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy: ", (sess.run(accuracy, feed_dict={x: test_x, y_: test_y})))
    pred_y = sess.run(y, feed_dict={x: test_x})
    mse = tf.reduce_mean(tf.square(pred_y - test_y))
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    accuracy = (sess.run(accuracy, feed_dict={x: train_x, y_: train_y}))
    accuracy_history.append(accuracy)
    
    print('epoch : ', epoch, ' - ', 'data: ', data, " - MSE: ", mse_, "- Train Accuracy: ", accuracy)

save_path = saver.save(sess, model_path)
print("Model dosyaya kaydedildi: %s" % save_path)

In [ ]:
plt.plot(accuracy_history)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Test Accuracy: ", (sess.run(accuracy, feed_dict={x: test_x, y_: test_y})))

pred_y = sess.run(y, feed_dict={x: test_x})
mse = tf.reduce_mean(tf.square(pred_y - test_y))
print("MSE: %.4f" % sess.run(mse))